In [29]:
import os
import sys
custom_paths = ['../src/'] # , '../cifar-10.1-pytorch/', '../data-copying/', '../PyTorch_CIFAR10/']
for path in custom_paths:
    if os.path.abspath(path) not in sys.path:
        sys.path.insert(0, os.path.abspath(path))
        print(f"Added {os.path.abspath(path)} to path")

Added /home/andrewbai/PyTorch-StudioGAN/src to path


In [86]:
import hnswlib
import numpy as np
import pickle
import glob
from time import time
from tqdm.notebook import tqdm, trange
from importlib import reload
from timeit import default_timer

import torch
import torch.nn as nn
from torch.nn import Parameter as P
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torchmetrics

In [91]:
import data_utils.load_dataset
reload(data_utils.load_dataset)
from data_utils.load_dataset import LoadDataset

from metrics.feat_extract_models import alexnet, inception, vgg

import misc

In [14]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = 'cuda'
torch.backends.cudnn.benchmark = True

In [3]:
dim = 2048
num_elements = 1000

In [4]:
# Generating sample data
data = np.float32(np.random.random((num_elements, dim)))
ids = np.arange(num_elements)

In [5]:
# Declaring index
p = hnswlib.Index(space='cosine', dim=dim) # possible options are l2, cosine or ip

In [6]:
# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements=num_elements, ef_construction=200, M=16)

In [7]:
# Element insertion (can be called several times):
p.add_items(data, ids)

In [8]:
# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

In [9]:
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
ts = []
for _ in range(100):
    start_t = time()
    labels, distances = p.knn_query(data, k = 1)
    ts.append(time() - start_t)
print(np.mean(ts))

0.013609724044799805


In [10]:
ts = []
for _ in range(100):
    start_t = time()
    np.min(np.matmul((data / np.linalg.norm(data, axis=1, keepdims=True)),
                           (data / np.linalg.norm(data, axis=1, keepdims=True)).T), axis=0)
    ts.append(time() - start_t)
print(np.mean(ts))

0.06000311136245728


Verdict: if we do it classwise then we can significantly speed it up. In that case, hnsw may not be significantly faster.

### cache ImageNet classwise embedding

In [47]:
proj_model = inception.WrappedInception3().eval().to(device)

##### trainable parameters

#####


In [71]:
cache_dir = '../data/imagenet_train_inception_classwise_embeddings'
os.makedirs(cache_dir, exist_ok=True)

In [17]:
%ls ../data/ILSVRC2012/train/

n01440764/  n02089973/  n02346627/  n03045698/  n03788195/  n04355338/
n01443537/  n02090379/  n02356798/  n03047690/  n03788365/  n04355933/
n01484850/  n02090622/  n02361337/  n03062245/  n03791053/  n04356056/
n01491361/  n02090721/  n02363005/  n03063599/  n03792782/  n04357314/
n01494475/  n02091032/  n02364673/  n03063689/  n03792972/  n04366367/
n01496331/  n02091134/  n02389026/  n03065424/  n03793489/  n04367480/
n01498041/  n02091244/  n02391049/  n03075370/  n03794056/  n04370456/
n01514668/  n02091467/  n02395406/  n03085013/  n03796401/  n04371430/
n01514859/  n02091635/  n02396427/  n03089624/  n03803284/  n04371774/
n01518878/  n02091831/  n02397096/  n03095699/  n03804744/  n04372370/
n01530575/  n02092002/  n02398521/  n03100240/  n03814639/  n04376876/
n01531178/  n02092339/  n02403003/  n03109150/  n03814906/  n04380533/
n01532829/  n02093256/  n02408429/  n03110669/  n03825788/  n04389033/
n01534433/  n02093428/  n02410509/  n03124043/  n03832673/  n04392985/
n01537

In [67]:
for class_name in sorted(os.listdir('../data/ILSVRC2012/train/')):
    print(class_name, len(os.listdir(os.path.join('../data/ILSVRC2012/train/', class_name))))

n01440764 1300
n01443537 1300
n01484850 1300
n01491361 1300
n01494475 1300
n01496331 1300
n01498041 1300
n01514668 1300
n01514859 1300
n01518878 1300
n01530575 1300
n01531178 1300
n01532829 1300
n01534433 1300
n01537544 1300
n01558993 1300
n01560419 1300
n01580077 1300
n01582220 1300
n01592084 1300
n01601694 1300
n01608432 1300
n01614925 1300
n01616318 1300
n01622779 1300
n01629819 1300
n01630670 1300
n01631663 1300
n01632458 1300
n01632777 1300
n01641577 1300
n01644373 1300
n01644900 1300
n01664065 1300
n01665541 1300
n01667114 1300
n01667778 1300
n01669191 1300
n01675722 1300
n01677366 1300
n01682714 1300
n01685808 1300
n01687978 1300
n01688243 1117
n01689811 1300
n01692333 1300
n01693334 1300
n01694178 1300
n01695060 1300
n01697457 1300
n01698640 1300
n01704323 1266
n01728572 1300
n01728920 1300
n01729322 1300
n01729977 1300
n01734418 1300
n01735189 1300
n01737021 1300
n01739381 1300
n01740131 1300
n01742172 1300
n01744401 1071
n01748264 1300
n01749939 1300
n01751748 1300
n01753488 

n04467665 1300
n04476259 1300
n04479046 1053
n04482393 1300
n04483307 1300
n04485082 1160
n04486054 1300
n04487081 1300
n04487394 1300
n04493381 1300
n04501370 1300
n04505470 1300
n04507155 1300
n04509417 1300
n04515003 1300
n04517823 1300
n04522168 1300
n04523525 1300
n04525038 1062
n04525305 1300
n04532106 1300
n04532670 1300
n04536866 1300
n04540053 1300
n04542943 1137
n04548280 1299
n04548362 1300
n04550184 1300
n04552348 1300
n04553703 1300
n04554684 1300
n04557648 1300
n04560804 1300
n04562935 1300
n04579145 1055
n04579432 1300
n04584207 1300
n04589890 1300
n04590129 1300
n04591157 1213
n04591713 1300
n04592741 1300
n04596742 1300
n04597913 1300
n04599235 1300
n04604644 1300
n04606251 1300
n04612504 1206
n04613696 1300
n06359193 1300
n06596364 1300
n06785654 1300
n06794110 1300
n06874185 1300
n07248320 1154
n07565083 1300
n07579787 1300
n07583066 1300
n07584110 1207
n07590611 1149
n07613480 1300
n07614500 1300
n07615774 1300
n07684084 1300
n07693725 1300
n07695742 1300
n07697313 

In [54]:
imagenet_dset_train = LoadDataset('imagenet', data_path='../data/', train=True, download=False, 
                                  resize_size=128, force_eval_mode=True)

In [55]:
imagenet_dl_train = DataLoader(imagenet_dset_train, batch_size=256, shuffle=False, 
                               pin_memory=False, num_workers=16, drop_last=False)

In [56]:
metric = torchmetrics.Accuracy().to(device)
embs, ys = [], []

with torch.no_grad():
    for x, y in tqdm(imagenet_dl_train, leave=False):
        x = x.to(device) # expect value range = [-1, 1]
        y = y.to(device)
        embs_, logits_ = proj_model(x)
        probs_ = F.softmax(logits_, dim=1)
        metric(probs_, y)

        embs.append(embs_.detach().cpu().numpy())
        ys.append(y.detach().cpu().numpy())

embs = np.concatenate(embs, axis=0)
ys = np.concatenate(ys, axis=0)
print(metric.compute().item())
embs.shape, ys.shape

  0%|          | 0/5005 [00:00<?, ?it/s]

0.796186625957489


((1281167, 2048), (1281167,))

In [69]:
for i, class_name in enumerate(sorted(os.listdir('../data/ILSVRC2012/train/'))):
    assert(len(os.listdir(os.path.join('../data/ILSVRC2012/train/', class_name))) == (ys == i).sum())
print('ok')

ok


In [72]:
for i, class_name in tqdm(enumerate(sorted(os.listdir('../data/ILSVRC2012/train/'))), leave=False):
    np.save(os.path.join(cache_dir, f"{class_name}.npy"), embs[ys == i])

0it [00:00, ?it/s]

### Build HNSW

In [73]:
# Declaring index
p = hnswlib.Index(space='cosine', dim=2048) # possible options are l2, cosine or ip

In [74]:
# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements=1281167, ef_construction=200, M=16)

In [79]:
# Element insertion (can be called several times):
cnt = 0
for npy_fpath in tqdm(sorted(glob.glob('../data/imagenet_train_inception_classwise_embeddings/*.npy')), 
                      leave=False):
    embs_ = np.load(npy_fpath)
    ids = np.arange(cnt, cnt + embs_.shape[0])
    p.add_items(embs_, ids)
    cnt += embs_.shape[0]

  0%|          | 0/1000 [00:00<?, ?it/s]

### Compare runtime and memory usage of classwise NND (gpu) vs HNSW (cpu)

In [80]:
imagenet_dset_valid = LoadDataset('imagenet', data_path='../data/', train=False, download=False, 
                                  resize_size=128, force_eval_mode=True)

In [81]:
imagenet_dl_valid = DataLoader(imagenet_dset_valid, batch_size=256, shuffle=False, 
                               pin_memory=False, num_workers=16, drop_last=False)

In [88]:
metric = torchmetrics.Accuracy().to(device)
embs_valid, ys_valid = [], []

with torch.no_grad():
    for x, y in tqdm(imagenet_dl_valid, leave=False):
        x = x.to(device) # expect value range = [-1, 1]
        y = y.to(device)
        embs_, logits_ = proj_model(x)
        probs_ = F.softmax(logits_, dim=1)
        metric(probs_, y)

        embs_valid.append(embs_.detach())
        ys_valid.append(y.detach())

embs_valid = torch.cat(embs_valid, dim=0)
ys_valid = torch.cat(ys_valid, dim=0)
print(metric.compute().item())
embs.shape, ys.shape

  0%|          | 0/196 [00:00<?, ?it/s]

0.6918200254440308


((1281167, 2048), (1281167,))

In [85]:
classwise_emb_tensors = []
for npy_fpath in tqdm(sorted(glob.glob('../data/imagenet_train_inception_classwise_embeddings/*.npy')), 
                      leave=False):
    embs_ = np.load(npy_fpath)
    classwise_emb_tensor = torch.from_numpy(embs_).to(device)
    classwise_emb_tensors.append(classwise_emb_tensor)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [98]:
bsize = 64
rts = []
n_repeat = 20
for _ in range(n_repeat):
    start = default_timer()
    for n_class in range(1000):
        class_embs = embs_valid[ys_valid == n_class]
        for i in np.arange(0, class_embs.shape[0], bsize):
            embs_ = class_embs[i:i+bsize].to(device)
            misc.calculate_knnd_torch(target_feats=embs_, 
                                      ref_feats=classwise_emb_tensors[n_class], return_indices=True)
    stop = default_timer()
    rts.append(stop - start)
print(np.mean(rts))

0.30661183341871945


In [99]:
bsize = 64
rts = []
n_repeat = 20
for _ in range(n_repeat):
    start = default_timer()
    for n_class in range(1000):
        class_embs = embs_valid[ys_valid == n_class]
        for i in np.arange(0, class_embs.shape[0], bsize):
            embs_ = class_embs[i:i+bsize].to(device)
            embs_ = embs_.cpu().numpy()
            p.knn_query(embs_, k=1)
    stop = default_timer()
    rts.append(stop - start)
print(np.mean(rts))

18.166192386765033


verdict: let's do conditional rejection then

### todo:
* add param to turn on conditional rejection
* implement sampler to only return batches of the same class
* implement rejector to rejection conditionally (possible refactoring)